In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [27]:
SUSCEPTIBLE = 0
INFECTED = 1
RECOVERED = 2

def init(N):
    L = np.sqrt(N)
    
    if not L.is_integer():
        raise 'N must be a square'
    
    cells = np.empty(N)
    cells.fill(SUSCEPTIBLE)
    cells[0] = INFECTED
    
    neighbors_list = []
    for i in range(N):
        up = i + 1
        down = i - 1
        left = i - L
        right = i + L
        
        if i % L == 0:
            down = i + L - 1
        if (i + 1) % L == 0:
            up = i - L + 1
        if i < L:
            left = N - L + i
        if i < N and i >= N - L:
            right = i - (N - L)
            
        neighbors_list.append([up, right, down, left])
        
    return cells, neighbors_list

In [28]:
N = 5*5

cells, neighbors_list = init(N)

prob_contamination = 0.1
prob_recovery = 0.1

susceptible_count = cells.count(SUSCEPTIBLE)
infected_count = cells.count(INFECTED)
recovered_count = cells.count(RECOVERED)

while infected_count != N and infected_count != 0:
    for i in range(N):
        if (i != INFECTED):
            continue

        neighbors = neighbors_list[i]

        for neighbor in neighbors:
            if (i != INFECTED)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
